In [2]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("../data/raw/application_record.zip")

In [4]:
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
df.describe()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS
count,4.385570e+05,438557.000000,4.385570e+05,438557.000000,438557.000000,438557.0,438557.000000,438557.000000,438557.000000,438557.000000
mean,6.022176e+06,0.427390,1.875243e+05,-15997.904649,60563.675328,1.0,0.206133,0.287771,0.108207,2.194465
std,5.716370e+05,0.724882,1.100869e+05,4185.030007,138767.799647,0.0,0.404527,0.452724,0.310642,0.897207
min,5.008804e+06,0.000000,2.610000e+04,-25201.000000,-17531.000000,1.0,0.000000,0.000000,0.000000,1.000000
25%,5.609375e+06,0.000000,1.215000e+05,-19483.000000,-3103.000000,1.0,0.000000,0.000000,0.000000,2.000000
50%,6.047745e+06,0.000000,1.607805e+05,-15630.000000,-1467.000000,1.0,0.000000,0.000000,0.000000,2.000000
75%,6.456971e+06,1.000000,2.250000e+05,-12514.000000,-371.000000,1.0,0.000000,1.000000,0.000000,3.000000
max,7.999952e+06,19.000000,6.750000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000


In [6]:
df["AGE"] = -df["DAYS_BIRTH"] / 365.0

In [7]:
df["DAYS_EMPLOYED"] = -df["DAYS_EMPLOYED"]

In [8]:
df = df.dropna()

In [9]:
from random import random

def calculate_approval(row):
    p = 1.0

    if row["AGE"] < 25:
        p = p - 0.2
    elif row["AGE"] >= 25 and row["AGE"] < 50:
        p = p - 0.1

    if row["CNT_CHILDREN"] == 0:
        p = p - 0.05
    elif row["CNT_CHILDREN"] > 0 and row["CNT_CHILDREN"] < 2:
        p = p - 0.1
    else:
        p = p - 0.15

    if row["AMT_INCOME_TOTAL"] < 100000:
        p = p - 0.4
    elif row["AMT_INCOME_TOTAL"] >= 100000 and row["AMT_INCOME_TOTAL"] < 200000:
        p = p - 0.2
    elif row["AMT_INCOME_TOTAL"] >= 200000 and row["AMT_INCOME_TOTAL"] < 300000:
        p = p - 0.1

    if row["DAYS_EMPLOYED"] < 365:
        p = p - 0.2
    elif row["DAYS_EMPLOYED"] >= 365 and row["DAYS_EMPLOYED"] < 2000:
        p = p - 0.1

    if row["FLAG_OWN_REALTY"] == 0.0:
        p = p - 0.06

    if row["FLAG_OWN_CAR"] == 0.0:
        p = p - 0.025

    return random() < p

In [10]:
df["APPROVED"] = df.apply(calculate_approval, axis=1)

In [11]:
# filter out extreme income values
df = df[df["AMT_INCOME_TOTAL"] < 1e6]

In [12]:
df.loc[:, ("APPROVED")] = df.loc[:, ("APPROVED")].eq(True).mul(1)
df.loc[:, ("FLAG_OWN_CAR")] = df.loc[:, ("FLAG_OWN_CAR")].eq("Y").mul(1)
df.loc[:, ("FLAG_OWN_REALTY")] = df.loc[:, ("FLAG_OWN_REALTY")].eq("Y").mul(1)

In [13]:
inputs = df[
        [
            "FLAG_OWN_CAR",
            "FLAG_OWN_REALTY",
            "CNT_CHILDREN",
            "AMT_INCOME_TOTAL",
            "AGE",
            "DAYS_EMPLOYED",
            "FLAG_WORK_PHONE",
        ]
    ]

In [14]:
inputs

,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,DAYS_EMPLOYED,FLAG_WORK_PHONE
2,1,1,0,112500.0,58.832877,1134,0
3,0,1,0,270000.0,52.356164,3051,0
4,0,1,0,270000.0,52.356164,3051,0
5,0,1,0,270000.0,52.356164,3051,0
6,0,1,0,270000.0,52.356164,3051,0
...,...,...,...,...,...,...,...
438541,0,1,0,202500.0,37.013699,2309,1
438548,1,1,1,135000.0,34.435616,2095,0
438553,0,0,0,103500.0,43.668493,3007,0
438554,0,0,0,54000.0,22.380822,372,1
